In [252]:
import pandas as pd
import json
from statsmodels.stats.inter_rater import fleiss_kappa,aggregate_raters
import numpy as np
from collections import Counter
from scipy import stats
import krippendorff
import os


In [253]:
dir = '../phase2_annotations_part1'

In [254]:
annotation_files = [file for file in os.listdir(dir) if file.endswith('.json')]
annotation_files

['grammatical_role-phaseTwo1103-coreference_resolution-batch2.json',
 'grammatical_role-phaseTwo1104-sentiment_analysis-batch1.json',
 'negation-phaseTwo1103-coreference_resolution-batch2.json',
 'grammatical_role-phaseTwo1031-dialog_contradiction_detection-batch5.json',
 'negation-phaseTwo1028-dialog_contradiction_detection-batch6.json',
 'negation-phaseTwo1028-dialog_contradiction_detection-batch7.json',
 'negation-phaseTwo1021-sentiment_analysis-batch1.json',
 'grammatical_role-phaseTwo1031-dialog_contradiction_detection-batch4.json',
 'negation-phaseTwo1103-coreference_resolution-batch3.json',
 'grammatical_role-phaseTwo1103-coreference_resolution-batch3.json',
 'negation-phaseTwo1103-coreference_resolution-batch4.json',
 'grammatical_role-phaseTwo1031-dialog_contradiction_detection-batch3.json',
 'negation-phaseTwo1021-sentiment_analysis-batch6.json',
 'grammatical_role-phaseTwo1103-coreference_resolution-batch4.json',
 'negation-phaseTwo1103-coreference_resolution-batch8.json',
 

In [255]:
test_types = set()
for file in annotation_files:
    task = file.split('-')[1]
    task = task.replace('.json', '')
    # print(task)
    test_types.add(task)
len(test_types)

5

In [256]:
annotations_all = {}
for file in annotation_files:
    fn = os.path.join(dir, file)
    with open(fn, 'rb') as f:
        batch_name = file.split('.json')[0]
        annotations = json.load(f)
        obj = {batch_name: annotations}
        annotations_all.update(obj)

In [257]:
len(annotations_all)

41

In [258]:
tasks_name = ['conversational_qa', 'coreference_resolution', 'dialogue_contradiction_detection', 'named_entity_recognition', 'sentiment_analysis']

In [259]:
modification_name = ['active_to_passive', 'capitalization', 'casual','compound_word', 'concept_replacement', 'coordinating_conjunction', 'derivation', 'dialectal', 'discourse', 'geographical_bias', 'grammatical_role', 'length_bias', 'negation', 'punctuation', 'sentiment', 'temporal_bias', 'typo_bias']

In [260]:
have_subtest = ['concept_replacement', 'dialectal', 'discourse', 'negation']

In [261]:
len(annotations_all['grammatical_role-phaseTwo1103-coreference_resolution-batch2']['5c16a14c5ca98a00018d30c4']['answers'])

23

In [262]:
len(annotations_all['grammatical_role-phaseTwo1103-coreference_resolution-batch2']['5fcfedeec803e31be21a9483']['answers'])

23

In [263]:
def construct_data(filter_list, filter = False):
    annotation_dict = {}
    for task in modification_name:
        
        annotation_dict[task] = []
        # test_name = task
        
    for batch_name in annotations_all.keys():
        # print(batch_name)
        task = batch_name.split('-')[0]
        bigtask_name = batch_name.split('-')[2]
        test_name = task 
        release = batch_name.split('-')[1]
        
        for task_name in modification_name:
            if task_name == task:
                
                df_task = []
                for annotator, value in annotations_all[batch_name].items():
                    if filter:
                        if annotator in filter_list:
                            continue
                    # for sample in annotator['answers']:
                    # print(annotator)
                    # print(value['answers'])
                    df = pd.DataFrame(value['answers'])
                    subtests = []
                    tests = [test_name]*len(df)
                    tasks = [bigtask_name]*len(df)
                    new_row = {}
                    for i, row in df.iterrows():
                        if row['is_control'] == True:
                            subtest = 'control'
                        else:
                            if release == 'release0808':
                                if task_name == 'concept_replacement':
                                    subtest = 'synonym'
                                elif task_name == 'discourse':
                                    subtest = 'reverse'
                                elif task_name == 'negation':
                                    subtest = 'verbal'
                                else:
                                    subtest = ''
                            else:
                                subtest = row['original_question'].get('type','')
                            if test_name not in have_subtest:
                                subtest = test_name
                        subtests.append(subtest)
                    if len(df) == 22:
                        worker_id = list(df['worker_id'])[0]
                        if df['is_control'].value_counts().get(True) < 3:
                            is_control = True
                            answer = 1
                            subtest = 'control'
                        else:
                            is_control = False
                            answer = 0
                        # if bigtask_name == 'active_to_passive':
                            # print(df)
                        new_row = {'time': None, 'instance_index': len(df)+1, 'worker_id': worker_id,	'explanation': None, 'answer': str(answer),	'original_question': 0,	'is_control':is_control, 'subtest': subtest, 'test': test_name, 'task': bigtask_name}
                        
                    elif len(df) < 16:
                        print(task, annotator) #some batches with less than 12 questions
                        continue
                    df['test'] = tests
                    df['task'] = tasks
                    df['subtest'] = subtests
                    if len(df) == 22:
                        new_row_df = pd.DataFrame([new_row])
                        df = pd.concat([df,new_row_df], ignore_index = True)
                    df_task.append(df)
                if len(df_task) <= 1:
                    continue
                df_task = pd.concat(df_task)    
                annotation_dict[task_name].append(df_task)
        # break
    return annotation_dict


In [264]:
annotations = construct_data(filter_list = None)

negation 5d3658b84f14c3001641516a
grammatical_role 60d3d1d337c621ff5e7ba61b
negation 5f14d5cdb5d4c20e78dc1672
negation 66e121a779bdeaea83f64d48
negation 6674c5516994038c34e78f26
negation 63e52f8ab3688f45f842114c
negation 663068f52490bf6d13f95968


In [265]:
annotations['grammatical_role'][1]

,worker_id,explanation,answer,time,currentInstance,original_question,is_control,test,task,subtest,instance_index
0,63e5745dd2df02224ca678f4,The author says that it is a masterpiece and v...,Positive,2024-11-03 22:50:41,0.0,"{'idx': -1, 'original_text': '', 'label': 1, '...",True,grammatical_role,sentiment_analysis,control,NaN
1,63e5745dd2df02224ca678f4,According to the author the film has bad actin...,Negative,2024-11-03 22:52:38,1.0,"{'idx': 598, 'original_text': 'nonsensical , d...",False,grammatical_role,sentiment_analysis,grammatical_role,NaN
2,63e5745dd2df02224ca678f4,the author says the movie is a disaster with b...,Negative,2024-11-03 22:53:58,2.0,"{'idx': -1, 'original_text': '', 'label': 0, '...",True,grammatical_role,sentiment_analysis,control,NaN
3,63e5745dd2df02224ca678f4,The authors description is very vauge but the ...,Negative,2024-11-03 22:57:09,3.0,"{'idx': 787, 'original_text': 'a valueless kid...",False,grammatical_role,sentiment_analysis,grammatical_role,NaN
4,63e5745dd2df02224ca678f4,the author says that the movie has no humor an...,Neutral,2024-11-03 22:59:21,4.0,"{'idx': 546, 'original_text': 'on the heels of...",False,grammatical_role,sentiment_analysis,grammatical_role,NaN
...,...,...,...,...,...,...,...,...,...,...,...
18,669bef32e12b168add192c3f,"The sentence says the memory ""bubbles up"" fro...",Positive,2024-11-04 00:06:51,18.0,"{'idx': 62, 'original_text': 'the primitive fo...",False,grammatical_role,sentiment_analysis,grammatical_role,NaN
19,669bef32e12b168add192c3f,The sentence calls the filmmakers and others ...,Positive,2024-11-04 00:07:35,19.0,"{'idx': 679, 'original_text': 'filmmakers who ...",False,grammatical_role,sentiment_analysis,grammatical_role,NaN
20,669bef32e12b168add192c3f,"The sentence calls ""the importance of being ea...",Positive,2024-11-04 00:08:24,20.0,"{'idx': 739, 'original_text': 'oscar wilde 's ...",False,grammatical_role,sentiment_analysis,grammatical_role,NaN
21,669bef32e12b168add192c3f,"The dream is a ""nightmare"" that won't go away,...",Negative,2024-11-04 00:09:19,21.0,"{'idx': 88, 'original_text': 'fresnadillo 's d...",False,grammatical_role,sentiment_analysis,grammatical_role,NaN


In [266]:
annotations['grammatical_role'][2]['subtest'].value_counts()

subtest
grammatical_role    60
control             12
Name: count, dtype: int64

In [267]:
def find_modes_and_counts(arr):
    # print(len(arr))
    # for row in arr:
        # print(len(row))
    arr = list(arr)
    arr = np.array(arr)
    # print(arr)
    _, cols = arr.shape
    modes = []
    counts = []
    
    # Process each column separately
    for j in range(cols):
        values, value_counts = np.unique(arr[:, j], return_counts=True)
        max_count = value_counts.max()
        # Find all values that appear max times
        col_modes = values[value_counts == max_count]
        # print(col_modes[0])
        # print(max_count[0])
        
        modes.append(col_modes[0])
        counts.append(max_count)
    
    modes = np.array(modes)
    counts = np.array(counts)
    return modes, counts


In [268]:
def string_xor_logical(arr1, arr2):
    # Using logical_xor for element-wise comparison
    arr1 = np.asarray(arr1)
    arr2 = np.asarray(arr2)
    
    # Convert strings to boolean based on non-emptiness
    # bool1 = arr1 != ''
    # bool2 = arr2 != ''
    # print(bool1)
    # print(bool2)
    # Perform logical XOR
    return np.logical_xor(arr1, arr2)


In [269]:
def calculate_krippendof_for_each_batch(df_batch):
    annotation_data = []
    # print(df_batch)
    for group in df_batch.groupby('worker_id'):
        preds = np.array(group[1]['answer']).astype(str)
        annotation_data.append(preds)
    # print(annotation_data)

    data = np.array(annotation_data)
    # print(data)
    # Transpose the data
    # transposed_data = data.T

    # Transform the transposed data
    # aggregated_data, categories = aggregate_raters(transposed_data)
    # print(aggregated_data)
    # Calculate Fleiss' Kappa
    alpha = krippendorff.alpha(data, level_of_measurement='nominal')
    # kappa = fleiss_kappa(aggregated_data, method='randolph')
    # print(f"Krippendorff alpha: {alpha:.4f}")
    return alpha

In [270]:
def calculate_fleiss_kappa_for_each_batch(df_batch):
    annotation_data = []
    for group in df_batch.groupby('worker_id'):
        preds = np.array(group[1]['answer']).astype(str)
        annotation_data.append(preds)
    # print(len(annotation_data))
    # for row in annotation_data:
    #     print(len(row))
    data = np.array(annotation_data)

    # print(data)
    # print(data)
    # Transpose the data
    transposed_data = data.T

    # Transform the transposed data
    aggregated_data, categories = aggregate_raters(transposed_data)
    # print(aggregated_data)
    # Calculate Fleiss' Kappa
    kappa = fleiss_kappa(aggregated_data, method='randolph')
    # print(f"Fleiss' Kappa: {kappa:.4f}")
    return kappa

In [271]:
def get_majority(batch, df_performance = None):
    annotation_data = []
    annotation_data_no_control = []
    annotation_data_control_only = []
    cur_best_annotator_score = 0
    best_preds = None
    for group in batch.groupby('worker_id'):
        # print(df_annotator[df_annotator[['success_rate_control_only']] == group[0]])
        preds = list(group[1]['answer'])
        answers_no_control = group[1][group[1]['is_control'] != True]['answer'] 
        answers_control_only = group[1][group[1]['is_control'] == True]['answer'] 
        if df_performance.empty != True:
            # print(df_performance)
            # print(group[0])
            annotator_score = df_performance[df_performance['annotator_id'] == group[0]]['success_rate_control_only'].values[0]
            # print(annotator_score)
            if annotator_score > cur_best_annotator_score:
                best_preds = preds
                cur_best_annotator_score = annotator_score

        # answers_control_only = group[1][group[1]['is_control'] == True]['answer'] 


        annotation_data.append(preds)
        annotation_data_no_control.append(answers_no_control)
        annotation_data_control_only.append(answers_control_only)


    # print(annotation_data)
    # modes  = stats.mode(annotation_data,axis = 0,keepdims = True).mode[0]
    # counts   = stats.mode(annotation_data,axis = 0,keepdims = True).count[0]
    # print(annotation_data)
    modes, counts = find_modes_and_counts(annotation_data)
    # print(stats.mode(annotation_data_no_control, axis=0, keepdims = True))
    # print(annotation_data_no_control)
    # modes_no_control = stats.mode(annotation_data_no_control, axis=0, keepdims = True).mode[0]
    # counts_no_control = stats.mode(annotation_data_no_control, axis=0, keepdims = True).count[0]
    modes_no_control, counts_no_control = find_modes_and_counts(annotation_data_no_control)
    if df_performance.empty != True:
        for i, count in enumerate(counts):
            if count <= len(batch.groupby('worker_id')) // 2:
                modes[i] = best_preds[i]

    # modes_control_only = stats.mode(annotation_data_control_only, axis=0).mode[0]
    modes_control_only, counts_control_only = find_modes_and_counts(annotation_data_control_only)
    # modes_control_only = np.zeros(len(modes) - len(modes_no_control), dtype = int)
    
    # print(modes.mode[0])
    return modes, modes_no_control, modes_control_only, counts

In [272]:
def calculate_success_rate_majority(batch, df_performance = None):
    rates = []
    rates_no_control = []
    rates_control_only = []
    rates_gpt4 = []
    modes, modes_no_control, modes_control_only, majority_counts = get_majority(batch, df_performance)
    
    # print(modes)
    # print(modes_no_control)
    # print(modes_control_only)
    df_annotator = []
    # answers = np.array(batch.groupby('worker_id')[0][1]['answer'])
    # print(len(batch.groupby('worker_id')))
    majority = len(batch.groupby('worker_id')) // 2
    # annotator_counts = len(batch.groupby('worker_id'))
    retain_count = 0
    total_count = 0
    retain_samples = [None]*len(modes)
    # retain = False
    
    for j,group in enumerate(batch.groupby('worker_id')):
        # if j == 1:
            # print(len(group[1]))
        for i,sample in group[1].iterrows():
            if sample['is_control'] == False:
                item = {}
                if type(sample['original_question']) == int:
                        continue
                item['original_question'] = sample['original_question']                       
                item['task'] = sample['task']
                item['test'] = sample['test']
                item['subtest'] = sample['subtest']
                # item['label'] = sample['answer']
                item['label'] = modes[i]
                item['explanation'] = sample['explanation']
                retain_samples[i] = item
        retain = True
        answers = np.array(list(group[1]['answer']))
        answers_no_control = np.array(group[1][group[1]['is_control'] != True]['answer']) 
        answers_control_only = np.array(group[1][group[1]['is_control'] == True]['answer'])

        
        correct_answer = answers == modes
        success_rate = np.count_nonzero(correct_answer) / len(answers)

        # print('Answer', answers_no_control)
        # print('Modes', modes_no_control)
        # print(np.logical_xor(answers_no_control, modes_no_control))
        correct_answer_no_control =  answers_no_control == modes_no_control

        success_rate_no_control = np.count_nonzero(correct_answer_no_control) / len(answers_no_control)

        correct_answer_control_only = answers_control_only == modes_control_only
        success_rate_control_only = np.count_nonzero(correct_answer_control_only) / len(answers_control_only)

        # retain_count += np.count_nonzero(modes_no_control)
        # total_count += len(answers_no_control)

        success_rate_gpt4 = np.count_nonzero(modes_no_control) / len(modes_no_control)
        negative_count = np.count_nonzero(answers_no_control == 0) 
        # print('Success rate for annotator with control', group[0], success_rate)
        # print('Success rate for annotator without control', group[0], success_rate_no_control)
        # print('Success rate for annotator control only', group[0], success_rate_control_only)
        df_annotator_row = {}
        df_annotator_row['annotator_id'] = group[0]
        df_annotator_row['success_rate_with_control'] = success_rate
        df_annotator_row['success_rate_without_control'] = success_rate_no_control
        df_annotator_row['success_rate_control_only'] = success_rate_control_only
        df_annotator_row['negative_count'] = negative_count
        # df_annotator_row['counts'] = counts
        df_annotator.append(df_annotator_row)

        rates.append(success_rate)
        rates_no_control.append(success_rate_no_control)
        rates_control_only.append(success_rate_control_only)
        rates_gpt4.append(success_rate_gpt4)

    # print(majority_counts)
    # print(majority)
    majority_rate = (majority_counts>majority).sum()/ len(answers)
    # print(len(retain_samples))
    total_count = len(answers_no_control)
    retain_count = len(retain_samples)
    # retain_count = np.count_nonzero(modes_no_control)
    # print(total_count, retain_count)
    # print(retain_samples)
    retain_samples = [x for x in retain_samples if x is not None]

    return np.mean(rates), np.mean(rates_no_control), np.mean(rates_control_only), np.mean(rates_gpt4), majority_rate, retain_samples, total_count, retain_count, df_annotator


In [273]:
def calculate_all(filter_list, filter, hit_function, df_performance = None):
    df_task = []
    df_annotator  = []
    df_subtask = []
    df_retain_task = []
    annotation_dict = construct_data(filter_list, filter)
    for key,value in annotation_dict.items():
        # print(key)
        kappa_task = []
        krippendorff_task = []
        success_rate_task = []
        success_rate_task_no_control = []
        success_rate_task_control_only = []
        success_rate_gpt4_task = []
        majority_rate_task = []
        for i, batch in enumerate(value):
            # print(batch)
            # if i==0:
            test_name = key
            task_name = list(batch['task'])[0]
            subtest_name = list(batch['subtest'])
            for name in subtest_name:
                if name != 'control':
                    subtest_name = name
                    break
            print(task_name, test_name, subtest_name)
                
            
            kappa_batch = calculate_fleiss_kappa_for_each_batch(batch)
            krippendorff_batch = calculate_krippendof_for_each_batch(batch)
            if hit_function == 'majority':
                success_rate_batch, success_rate_batch_no_control, success_rate_batch_control_only, success_rate_gpt4_batch, majority_rate_batch, retain_samples_batch, total_count_batch, retain_count_batch,  df_annotator_row = calculate_success_rate_majority(batch, df_performance)
            # print('Fleiss kappa batch', i+1, kappa_batch)
            # print('Krippendorff alpha batch', i+1, krippendorff_batch)

            kappa_task.append(kappa_batch)
            krippendorff_task.append(krippendorff_batch)

            # print('Success rate batch with control', i+1, success_rate_batch)
            # print('Success rate batch without control', i+1, success_rate_batch_no_control)
            success_rate_task.append(success_rate_batch)
            success_rate_task_no_control.append(success_rate_batch_no_control)
            success_rate_task_control_only.append(success_rate_batch_control_only)
            success_rate_gpt4_task.append(success_rate_gpt4_batch)
            majority_rate_task.append(majority_rate_batch)
            # annotator_counts_task.append(annotator_counts_batch)
            df_annotator.extend(df_annotator_row) 
            # df_majority_counts.extend(majority_counts)
            # df_annotator_counts.append(annotator_counts_batch)
            df_retain_task.extend(retain_samples_batch)
            df_subtask_row = {}
            df_subtask_row['task'] = task_name
            df_subtask_row['test'] = test_name
            df_subtask_row['subtest'] = subtest_name
            df_subtask_row['total'] = total_count_batch
            df_subtask_row['retain'] = retain_count_batch
            df_subtask.append(df_subtask_row)
        # print('Fleiss kappa for', key, np.mean(kappa_task))
        # print('Krippendorff alpha for', key, np.mean(krippendorff_task))

        # print('Success rate task with control', key, np.mean(success_rate_task))
        # print('Success rate task without control', key, np.mean(success_rate_task_no_control))
        # print('Success rate task control only', key, np.mean(success_rate_task_control_only))

        df_task_row = {}
        df_task_row['task'] = key
        df_task_row['kappa'] = np.mean(kappa_task)
        df_task_row['krippendorff'] = np.mean(krippendorff_task)

        df_task_row['success_rate_with_control'] = np.mean(success_rate_task)
        df_task_row['success_rate_without_control'] = np.mean(success_rate_task_no_control)
        df_task_row['success_rate_control_only'] = np.mean(success_rate_task_control_only)
        df_task_row['success_rate_gpt4'] = np.mean(success_rate_gpt4_task)
        df_task_row['majority_rate'] = np.mean(majority_rate_task)
        # df_task_row['annotator_counts'] = sum(annotator_counts_task)
        # print('annotator count',df_annotator_counts)
        df_task.append(df_task_row)
    df_task = pd.DataFrame(data = df_task)
    df_subtask = pd.DataFrame(data = df_subtask)
    df_annotator = pd.DataFrame(data = df_annotator)
    # print(df_retain_task)
    df_retain_task = pd.DataFrame(data = df_retain_task)
    return df_task, df_annotator, df_retain_task, df_subtask

In [274]:
df_task, df_annotator, df_retain_task, df_subtask = calculate_all(filter_list = None, filter = False, hit_function = 'majority', df_performance = pd.DataFrame())

negation 5d3658b84f14c3001641516a
grammatical_role 60d3d1d337c621ff5e7ba61b
negation 5f14d5cdb5d4c20e78dc1672
negation 66e121a779bdeaea83f64d48
negation 6674c5516994038c34e78f26
negation 63e52f8ab3688f45f842114c
negation 663068f52490bf6d13f95968
coreference_resolution grammatical_role grammatical_role
sentiment_analysis grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
coreference_resolution grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
coreference_resolution grammatical_role grammatical_role
coreference_resolution grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
coreference_resolution grammatical_role grammatical_role
sentiment_analysis grammatical_role grammatical_role
sentiment_analysis grammatical_role gr

/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


dialog_contradiction_detection negation 
coreference_resolution negation 
sentiment_analysis negation verbal
coreference_resolution negation 
sentiment_analysis negation lexical
dialog_contradiction_detection negation 
dialog_contradiction_detection negation 
sentiment_analysis negation approximate
dialog_contradiction_detection negation 
coreference_resolution negation 
sentiment_analysis negation absolute
dialog_contradiction_detection negation 
dialog_contradiction_detection negation 


/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [275]:
df_task = df_task.dropna()

In [276]:
df_task = df_task.drop(columns=['success_rate_gpt4'])

In [277]:
df_task

,task,kappa,krippendorff,success_rate_with_control,success_rate_without_control,success_rate_control_only,majority_rate
10,grammatical_role,0.528048,0.465713,0.796827,0.791733,0.833333,0.798808
12,negation,0.449161,0.376827,0.763660,0.748958,0.847222,0.730303


In [278]:
df_annotator_group = df_annotator.groupby('annotator_id').mean().reset_index()
df_annotator_group['success_rate_control_only'].value_counts()

success_rate_control_only
1.000000    93
0.666667    44
0.333333     8
0.000000     4
Name: count, dtype: int64

In [279]:
df_retain_task['original_question'][0]

{'index_in_phase1_annotated_data': 29,
 'text': 'Vivan asked Lakshman to get him some ice cream because he was hot.',
 'pronoun': 'he',
 'candidates': ['Lakshman', 'Vivan'],
 'label': 'unknown',
 'label_antecedent': 'n/a',
 'is_control_question': False,
 'source': 'phase1 annotation'}

In [280]:
df_retain_task

,original_question,task,test,subtest,label,explanation
0,"{'index_in_phase1_annotated_data': 29, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Vivan,him rather than himself so clearly means Lakshman
1,"{'index_in_phase1_annotated_data': 33, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Steve,Jose could have hired Steve because Jose is a ...
2,"{'index_in_phase1_annotated_data': 36, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Wanda,"Regina is asking Wanda about an opinion, so it..."
3,"{'index_in_phase1_annotated_data': 23, 'text':...",coreference_resolution,grammatical_role,grammatical_role,The attackers,The journalists were waiting so couldn't arrive
4,"{'index_in_phase1_annotated_data': 30, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Stephanie,Theresa is dead so Logan wouldn't be talking t...
...,...,...,...,...,...,...
705,"{'index_in_original_testset': 2179, 'conversat...",dialog_contradiction_detection,negation,,No,Even though this statement looks like a contra...
706,"{'index_in_original_testset': 42, 'conversatio...",dialog_contradiction_detection,negation,,No,It appears that the statements are not contrad...
707,"{'index_in_original_testset': 2953, 'conversat...",dialog_contradiction_detection,negation,,No,Complete thought no contradiction noted.
708,"{'index_in_original_testset': 3382, 'conversat...",dialog_contradiction_detection,negation,,Hard to say,It appear that person B likes horses. I would ...


In [281]:
dialog_label_mapping = {0:'No', 1: 'Yes', 2: 'Hard to say'}
sentiment_label_mapping = {0: 'Negative', 1: 'Positive'}

In [282]:
rows = []
for i, row in df_retain_task.iterrows():
    task  = row['task']
    test = row['test']
    if task == 'coreference_resolution':
        task_dir = 'thinh'
    elif task == 'dialog_contradiction_detection':
        task_dir = 'rongxin'
    elif task == 'sentiment_analysis':
        task_dir = 'yulia'
    file_to_load = '../data_for_phase2/' + task_dir + '/' + test + '.json'
    df_original = pd.read_json(file_to_load)
    # print(row['original_question'])
    if row['original_question'].get('index_in_phase1_annotated_data')!=None:
        index = row['original_question']['index_in_phase1_annotated_data']
        # print(df_original.iloc[index])
        # if df_original.iloc[index].get('original_label')!=None:
        original_label = df_original.iloc[index]['original_label']
        candidates = df_original.iloc[index]['original_candidates']
        # print(candidates)
        original_label = candidates[original_label]

        # else:
            # original_label = df_original.iloc[index]['label']
            
    elif row['original_question'].get('index_in_original_testset')!=None:
        index = row['original_question']['index_in_original_testset']
        # print(index)
        # print(df_original)
        original_label = df_original.loc[df_original[0] == index][1].values[0]['label']
        original_label = dialog_label_mapping[original_label]
        # print()
        # print(original_label)
    else:
        # print(row['original_question'])
        index = row['original_question']['idx']
        original_label = df_original.loc[df_original['idx'] == index]['label'].values[0]
        original_label = sentiment_label_mapping[original_label]

        # print(original_label)
    row['original_label'] = original_label
    rows.append(row)


In [283]:
df_phase2  = pd.DataFrame(data = rows)

In [284]:
df_phase2[df_phase2['task'] == 'coreference_resolution']

,original_question,task,test,subtest,label,explanation,original_label
0,"{'index_in_phase1_annotated_data': 29, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Vivan,him rather than himself so clearly means Lakshman,Lakshman
1,"{'index_in_phase1_annotated_data': 33, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Steve,Jose could have hired Steve because Jose is a ...,Jose
2,"{'index_in_phase1_annotated_data': 36, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Wanda,"Regina is asking Wanda about an opinion, so it...",Regina
3,"{'index_in_phase1_annotated_data': 23, 'text':...",coreference_resolution,grammatical_role,grammatical_role,The attackers,The journalists were waiting so couldn't arrive,the journalists
4,"{'index_in_phase1_annotated_data': 30, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Stephanie,Theresa is dead so Logan wouldn't be talking t...,Theresa
...,...,...,...,...,...,...,...
655,"{'index_in_phase1_annotated_data': 10, 'text':...",coreference_resolution,negation,,Cannot Determine — Insufficient Information,He could be either,Steven
656,"{'index_in_phase1_annotated_data': 13, 'text':...",coreference_resolution,negation,,Tony,Sentence subject is Tony,Jeff
657,"{'index_in_phase1_annotated_data': 8, 'text': ...",coreference_resolution,negation,,Cannot Determine — Insufficient Information,Could be either,Dina
658,"{'index_in_phase1_annotated_data': 5, 'text': ...",coreference_resolution,negation,,Cannot Determine — Insufficient Information,Bat or ball could be it,The bat


In [285]:
df_phase2['label_change'] = (df_phase2['label'] != df_phase2['original_label']).astype(int)

In [286]:
df_phase2

,original_question,task,test,subtest,label,explanation,original_label,label_change
0,"{'index_in_phase1_annotated_data': 29, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Vivan,him rather than himself so clearly means Lakshman,Lakshman,1
1,"{'index_in_phase1_annotated_data': 33, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Steve,Jose could have hired Steve because Jose is a ...,Jose,1
2,"{'index_in_phase1_annotated_data': 36, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Wanda,"Regina is asking Wanda about an opinion, so it...",Regina,1
3,"{'index_in_phase1_annotated_data': 23, 'text':...",coreference_resolution,grammatical_role,grammatical_role,The attackers,The journalists were waiting so couldn't arrive,the journalists,1
4,"{'index_in_phase1_annotated_data': 30, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Stephanie,Theresa is dead so Logan wouldn't be talking t...,Theresa,1
...,...,...,...,...,...,...,...,...
705,"{'index_in_original_testset': 2179, 'conversat...",dialog_contradiction_detection,negation,,No,Even though this statement looks like a contra...,No,0
706,"{'index_in_original_testset': 42, 'conversatio...",dialog_contradiction_detection,negation,,No,It appears that the statements are not contrad...,Yes,1
707,"{'index_in_original_testset': 2953, 'conversat...",dialog_contradiction_detection,negation,,No,Complete thought no contradiction noted.,No,0
708,"{'index_in_original_testset': 3382, 'conversat...",dialog_contradiction_detection,negation,,Hard to say,It appear that person B likes horses. I would ...,No,1


In [287]:
df_phase2[df_phase2['task'] == 'coreference_resolution']

,original_question,task,test,subtest,label,explanation,original_label,label_change
0,"{'index_in_phase1_annotated_data': 29, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Vivan,him rather than himself so clearly means Lakshman,Lakshman,1
1,"{'index_in_phase1_annotated_data': 33, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Steve,Jose could have hired Steve because Jose is a ...,Jose,1
2,"{'index_in_phase1_annotated_data': 36, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Wanda,"Regina is asking Wanda about an opinion, so it...",Regina,1
3,"{'index_in_phase1_annotated_data': 23, 'text':...",coreference_resolution,grammatical_role,grammatical_role,The attackers,The journalists were waiting so couldn't arrive,the journalists,1
4,"{'index_in_phase1_annotated_data': 30, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Stephanie,Theresa is dead so Logan wouldn't be talking t...,Theresa,1
...,...,...,...,...,...,...,...,...
655,"{'index_in_phase1_annotated_data': 10, 'text':...",coreference_resolution,negation,,Cannot Determine — Insufficient Information,He could be either,Steven,1
656,"{'index_in_phase1_annotated_data': 13, 'text':...",coreference_resolution,negation,,Tony,Sentence subject is Tony,Jeff,1
657,"{'index_in_phase1_annotated_data': 8, 'text': ...",coreference_resolution,negation,,Cannot Determine — Insufficient Information,Could be either,Dina,1
658,"{'index_in_phase1_annotated_data': 5, 'text': ...",coreference_resolution,negation,,Cannot Determine — Insufficient Information,Bat or ball could be it,The bat,1


In [288]:
label_change_rate = (df_phase2.groupby(['task', 'test'])['label_change']
                    .mean()  # Calculate mean of label_change (will give us the rate)
                    .multiply(100)  # Convert to percentage
                    .round(2)  # Round to 2 decimal places
                    .reset_index())  # Convert from Series to DataFrame

# Display the results
print("Label Change Rate (%) by Task and Test:")
print(label_change_rate)

Label Change Rate (%) by Task and Test:
                             task              test  label_change
0          coreference_resolution  grammatical_role         81.58
1          coreference_resolution          negation         52.86
2  dialog_contradiction_detection  grammatical_role         38.89
3  dialog_contradiction_detection          negation         37.78
4              sentiment_analysis  grammatical_role         19.18
5              sentiment_analysis          negation         51.27


In [289]:
# Calculate label change statistics for each group
label_change_stats = (df_phase2.groupby(['task', 'test'])
                     .agg({
                         'label_change': ['size','sum', 'mean' ]  # mean for rate, sum for mismatches, size for total
                     })
                     .round(4))

# Flatten column names and reset index
label_change_stats.columns = ['total_samples', 'samples_with_label_change' ,'change_rate' ]
label_change_stats = label_change_stats.reset_index()

# Convert rate to percentage
label_change_stats['change_rate'] = label_change_stats['change_rate'] * 100

# Sort by change rate in descending order (optional)
label_change_stats = label_change_stats.sort_values('change_rate', ascending=False)

print("Label Change Statistics by Task and Test:")
print(label_change_stats)

Label Change Statistics by Task and Test:
                             task              test  total_samples  \
0          coreference_resolution  grammatical_role            114   
1          coreference_resolution          negation            140   
5              sentiment_analysis          negation            158   
2  dialog_contradiction_detection  grammatical_role             90   
3  dialog_contradiction_detection          negation            135   
4              sentiment_analysis  grammatical_role             73   

   samples_with_label_change  change_rate  
0                         93        81.58  
1                         74        52.86  
5                         81        51.27  
2                         35        38.89  
3                         51        37.78  
4                         14        19.18  


In [290]:
label_change_stats

,task,test,total_samples,samples_with_label_change,change_rate
0,coreference_resolution,grammatical_role,114,93,81.58
1,coreference_resolution,negation,140,74,52.86
5,sentiment_analysis,negation,158,81,51.27
2,dialog_contradiction_detection,grammatical_role,90,35,38.89
3,dialog_contradiction_detection,negation,135,51,37.78
4,sentiment_analysis,grammatical_role,73,14,19.18


In [291]:
filter_annotators = list(df_annotator_group.loc[df_annotator_group['success_rate_control_only'] < 0.5]['annotator_id'])

In [292]:
len(filter_annotators)

12

In [293]:
df_task_filter, df_annotator_filter, df_retain_task_filter, df_subtask_filter = calculate_all(filter_list = filter_annotators, filter = True, hit_function = 'majority', df_performance= df_annotator_group)

negation 5d3658b84f14c3001641516a
grammatical_role 60d3d1d337c621ff5e7ba61b
negation 5f14d5cdb5d4c20e78dc1672
negation 66e121a779bdeaea83f64d48
negation 6674c5516994038c34e78f26
negation 63e52f8ab3688f45f842114c
negation 663068f52490bf6d13f95968
coreference_resolution grammatical_role grammatical_role
sentiment_analysis grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
coreference_resolution grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
coreference_resolution grammatical_role grammatical_role
coreference_resolution grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
dialog_contradiction_detection grammatical_role grammatical_role
coreference_resolution grammatical_role grammatical_role
sentiment_analysis grammatical_role grammatical_role
sentiment_analysis grammatical_role gr

/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


dialog_contradiction_detection negation 
sentiment_analysis negation approximate
coreference_resolution negation 
coreference_resolution negation 
sentiment_analysis negation lexical
dialog_contradiction_detection negation 
sentiment_analysis negation verbal
coreference_resolution negation 
sentiment_analysis negation double
dialog_contradiction_detection negation 
coreference_resolution negation 
sentiment_analysis negation verbal
coreference_resolution negation 
sentiment_analysis negation lexical
dialog_contradiction_detection negation 
dialog_contradiction_detection negation 
sentiment_analysis negation approximate
dialog_contradiction_detection negation 
coreference_resolution negation 
sentiment_analysis negation absolute
dialog_contradiction_detection negation 
dialog_contradiction_detection negation 


/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hungthinht/miniconda3/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [414]:
rows = []
rows_coref = []
rows_sentiment = []
rows_dialog = []

for i, row in df_retain_task_filter.iterrows():
    task  = row['task']
    test = row['test']
    if task == 'coreference_resolution':
        task_dir = 'thinh'
    elif task == 'dialog_contradiction_detection':
        task_dir = 'rongxin'
    elif task == 'sentiment_analysis':
        task_dir = 'yulia'
    file_to_load = '../data_for_phase2/' + task_dir + '/' + test + '.json'
    df_original = pd.read_json(file_to_load)
    # print(row['original_question'])
    if row['original_question'].get('index_in_phase1_annotated_data')!=None:
        index = row['original_question']['index_in_phase1_annotated_data']
        # print(df_original.iloc[index])
        # if df_original.iloc[index].get('original_label')!=None:
        original_label = df_original.iloc[index]['original_label']
        candidates = df_original.iloc[index]['original_candidates']
        # print(candidates)
        original_label = candidates[original_label]
        original_row = df_original.iloc[index]
        original_row['modified_label'] = row['label']
        original_row['test'] = row['test']
        rows_coref.append(original_row)
        # else:
            # original_label = df_original.iloc[index]['label']
            
    elif row['original_question'].get('index_in_original_testset')!=None:
        index = row['original_question']['index_in_original_testset']
        # print(index)
        # print(df_original)
        original_label = df_original.loc[df_original[0] == index][1].values[0]['label']
        original_label = dialog_label_mapping[original_label]
        original_row = df_original.loc[df_original[0] == index][1].values[0]
        # print()
        original_row['modified_label'] = row['label']
        original_row['test'] = row['test']

        # print(original_label)
        rows_dialog.append(original_row)
    else:
        # print(row['original_question'])
        index = row['original_question']['idx']
        original_label = df_original.loc[df_original['idx'] == index]['label'].values[0]
        original_label = sentiment_label_mapping[original_label]
        original_row = df_original.loc[df_original['idx'] == index]
        original_row['modified_label'] = row['label']
        original_row['test'] = row['test']
        for i, item in original_row.iterrows():
            print(item)
            rows_sentiment.append(item)
        
        # print(original_label)
    row['original_label'] = original_label

    
    rows.append(row)


/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['modified_label'] = row['label']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['test'] = row['test']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

idx                                                             598
original_text     nonsensical , dull `` cyber-horror '' flick is...
label                                                             0
modified_text     Flat scares and bad acting, dull ``cyber-horro...
type                                               grammatical_role
modified_label                                             Negative
test                                               grammatical_role
Name: 9, dtype: object
idx                                                             787
original_text     a valueless kiddie paean to pro basketball und...
label                                                             0
modified_text     a valueless kiddie paean to the nba underwritt...
type                                               grammatical_role
modified_label                                             Negative
test                                               grammatical_role
Name: 10, dtype: object
i

/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['modified_label'] = row['label']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['test'] = row['test']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

idx                                                             570
original_text     seldahl 's barbara is a precise and moving por...
label                                                             1
modified_text     someone's barbara is a precise and moving port...
type                                               grammatical_role
modified_label                                             Positive
test                                               grammatical_role
Name: 68, dtype: object
idx                                                             748
original_text     and when you 're talking about a slapstick com...
label                                                             0
modified_text     and when a slapstick comedy is talking about y...
type                                               grammatical_role
modified_label                                             Negative
test                                               grammatical_role
Name: 34, dtype: object


/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['modified_label'] = row['label']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['test'] = row['test']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

...

idx                                                             679
original_text     filmmakers who can deftly change moods are tre...
label                                                             1
modified_text     filmmakers who cannot deftly change moods are ...
type                                                         verbal
modified_label                                             Positive
test                                                       negation
Name: 127, dtype: object
idx                                                             853
original_text     ( serry ) wants to blend politics and drama , ...
label                                                             1
modified_text     ( serry ) wants to separate politics and drama...
type                                                        lexical
modified_label                                             Positive
test                                                       negation
Name: 120, dtype: objec

/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['modified_label'] = row['label']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['test'] = row['test']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

idx                                                             283
original_text     a rewarding work of art for only the most pati...
label                                                             1
modified_text     a rewarding work of art for only the most pati...
type                                                         verbal
modified_label                                             Negative
test                                                       negation
Name: 154, dtype: object
idx                                                      179
original_text      the film 's performances are thrilling . 
label                                                      1
modified_text     the film's performances are not thrilling.
type                                                  verbal
modified_label                                      Negative
test                                                negation
Name: 149, dtype: object
idx                                            

/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['modified_label'] = row['label']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_row['test'] = row['test']
/var/folders/10/sw0slq6565ddbg_lqxxhpqzh0000gn/T/ipykernel_8912/3507174061.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

idx                                                             528
original_text     it haunts , horrifies , startles and fascinate...
label                                                             1
modified_text     it haunts, horrifies, startles and fascinates;...
type                                                    approximate
modified_label                                             Positive
test                                                       negation
Name: 22, dtype: object
idx                                                             528
original_text     it haunts , horrifies , startles and fascinate...
label                                                             1
modified_text     it haunts, horrifies, startles and fascinates;...
type                                                       absolute
modified_label                                             Positive
test                                                       negation
Name: 58, dtype: object


In [418]:
df_coref = pd.DataFrame(data = rows_coref)
df_coref

,original_text,original_candidates,original_pronoun,original_label,modified_text,type,modified_candidates,modified_pronoun,modified_label,test
29,Lakshman asked Vivan to get him some ice cream...,"[Lakshman, Vivan]",he,0,Vivan asked Lakshman to get him some ice cream...,grammatical_role,"[Vivan, Lakshman]",he,Vivan,grammatical_role
33,Steve hired Jose because he was a hard worker.,"[Steve, Jose]",he,1,Jose hired Steve because he was a hard worker.,grammatical_role,"[Jose, Steve]",he,Steve,grammatical_role
36,Wanda asks Regina if she wants to go with him ...,"[Wanda, Regina]",she,1,"Regina asks Wanda if she wants to go with him,...",grammatical_role,"[Regina, Wanda]",she,Wanda,grammatical_role
23,The attackers had been waiting for the journal...,"[The attackers, the journalists]",they,1,The journalists had been waiting for the attac...,grammatical_role,"[The journalists, the attackers]",they,The attackers,grammatical_role
30,"Theresa almost admits to killing Stephanie , b...","[Theresa, Stephanie]",her,0,"Stephanie almost admits to killing Theresa, bu...",grammatical_role,"[Stephanie, Theresa]",her,Stephanie,grammatical_role
...,...,...,...,...,...,...,...,...,...,...
10,Joe raced against Steven because he would not ...,"[Joe, Steven]",he,1,Joe did not race against Steven because he wou...,verbal,"[Joe, Steven]",he,Cannot Determine — Insufficient Information,negation
13,Tony helped Jeff because he needed help.,"[Tony, Jeff]",he,1,Tony did not help Jeff because he needed help.,verbal,"[Tony, Jeff]",he,Tony,negation
8,Rebecca kissed Dina because she is a sweet girl.,"[Rebecca, Dina]",she,1,Rebecca did not kiss Dina because she is a swe...,verbal,"[Rebecca, Dina]",she,Dina,negation
5,The bat hit the ball because it flew in the wa...,"[The bat, the ball]",it,0,The bat did not hit the ball because it flew i...,verbal,"[The bat, the ball]",it,Cannot Determine — Insufficient Information,negation


In [420]:
df_sentiment = pd.DataFrame(data = rows_sentiment)
df_sentiment

,idx,original_text,label,modified_text,type,modified_label,test
9,598,"nonsensical , dull `` cyber-horror '' flick is...",0,"Flat scares and bad acting, dull ``cyber-horro...",grammatical_role,Negative,grammatical_role
10,787,a valueless kiddie paean to pro basketball und...,0,a valueless kiddie paean to the nba underwritt...,grammatical_role,Negative,grammatical_role
5,546,on the heels of the ring comes a similarly mor...,1,on the heels of a similarly morose and humorle...,grammatical_role,Neutral,grammatical_role
3,269,with virtually no interesting elements for an ...,0,with virtually no interesting elements for a t...,grammatical_role,Negative,grammatical_role
16,551,nothing in waking up in reno ever inspired me ...,0,nothing in waking up in its inhabitants ever i...,grammatical_role,Negative,grammatical_role
...,...,...,...,...,...,...,...
128,269,with virtually no interesting elements for an ...,0,with virtually no interesting elements for an ...,verbal,Negative,negation
17,196,"so , too , is this comedy about mild culture c...",1,"so, too, is this comedy about mild culture cla...",approximate,Negative,negation
48,196,"so , too , is this comedy about mild culture c...",1,"so, too, is this comedy about mild culture cla...",absolute,Negative,negation
9,224,"nothing 's at stake , just a twisty double-cro...",1,"nothing's at stake, just a twisty double-cross...",approximate,Negative,negation


In [421]:
df_dialog = pd.DataFrame(data = rows_dialog)
df_dialog

,conversation_id,agents,conversation_contexts,auxiliary,turns,is_truncated,record_id,is_contradiction,writer_contradiction_indices,aggregated_contradiction_indices,num_of_turns_by_writer,dialogue,label,original_text,modified_text,type,modified_label,test
0,e4426e3f-dec2-4485-8b08-770e8f2707f8##hit:1036...,"{'1': {'is_human': True, 'persona_lines': ''},...",sentimental ## I recently remembered a sweet m...,{'source': 'EMPATHY_test'},"[{'turn_id': 0, 'agent_id': 0, 'text': 'I rece...",True,21aa9bf1-26c0-413e-8341-d4c08ce25735,True,"[2, 4, 6]","[2, 4, 6]",2.0,[agent 0: I recently remembered a sweet memory...,1,"agent 0: Amazingly selfish, she never spent a ...","Amazingly selfish, nobody besides herself ever...",grammatical_role,Yes,grammatical_role
1,5a54e4d6-263d-44bc-a3c5-b152bb21ff00#truncated#8,"{'1': {'is_human': True, 'persona_lines': ['i ...",About to open a restaurant with my friend. I h...,{'source': 'BST_test'},"[{'turn_id': 0, 'agent_id': 0, 'text': 'About ...",True,8ac99589-9a17-460a-b51c-4817cb97cea5,False,None,[],NaN,[agent 0: About to open a restaurant with my f...,0,agent 1: That is good to hear I would enjoy a ...,That is good to hear a night would enjoy some ...,grammatical_role,No,grammatical_role
2,9e861f18-60c0-4262-a2d6-be2cd742959c#truncated#11,"{'1': {'is_human': True, 'persona_lines': ['i ...",None,{'source': 'BST_test'},"[{'turn_id': 0, 'agent_id': 0, 'text': 'what d...",True,ab3b412f-2bbc-4d0f-a3ba-47652b84f549,True,"[4, 6, 8, 12]","[4, 6, 8, 12]",2.0,[agent 0: what do you do for fun ? i love long...,1,"agent 0: Yes, I joined the TV production club ...","Yes, I joined the journalism major club becaus...",grammatical_role,Yes,grammatical_role
3,c4a13954-a9cf-4ddc-862b-1b2ef844fc09#truncated#9,"{'1': {'is_human': True, 'persona_lines': ['i ...",Wall Street,{'source': 'BST_test'},"[{'turn_id': 0, 'agent_id': 0, 'text': 'I didn...",True,e545bd06-6724-4552-81f3-c77de1c2643c,False,None,[],NaN,[agent 0: I didn't realize it was that large. ...,0,agent 0: She's a poodle! They're a very smart ...,She's a very smart breed! They're a poodle. Wh...,grammatical_role,No,grammatical_role
4,7cd2da6a-3622-4d07-8eba-546888edb085#truncated#12,"{'1': {'is_human': True, 'persona_lines': ['i ...",None,{'source': 'BST_test'},"[{'turn_id': 0, 'agent_id': 0, 'text': 'milk m...",True,e993b321-e564-4127-975d-3fba2b315bf3,False,None,[],NaN,"[agent 0: milk makes strong bones though , how...",0,"agent 1: Olive oil. If you like nuts, they ar...","Cashews or walnuts. If you like olive oil, the...",grammatical_role,No,grammatical_role
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,78123d1f-6d5b-47be-9749-178228ce5c04##hit:310_...,"{'1': {'is_human': True, 'persona_lines': ''},...",confident ## I went in for a third interview a...,{'source': 'EMPATHY_test'},"[{'turn_id': 0, 'agent_id': 0, 'text': 'I know...",True,16f03442-b751-4f44-9128-fce009b359ac,False,None,[],NaN,[agent 0: I know I shouldn't count my chicks b...,0,agent 0: This advertising company! The owner w...,This advertising company! The owner was there ...,lexical,No,negation
221,777e0957-354a-47ce-b33a-abf4e34e2e6a##hit:6435...,"{'1': {'is_human': True, 'persona_lines': ''},...",proud ## My sister graduated from college afte...,{'source': 'EMPATHY_test'},"[{'turn_id': 0, 'agent_id': 0, 'text': 'My sis...",False,980045a7-fe92-4546-8d1c-bf7d418b50c1,True,"[3, 5]","[3, 5]",2.0,[agent 0: My sister graduated from college aft...,1,agent 1: Young peoples never give up on there ...,Young peoples always abandon their dreams.,lexical,No,negation
222,6a9e5e48-dd87-4416-af2a-4819b6f3e764##hit:7415...,"{'1': {'is_human': True, 'persona_lines': ''},...",caring ## My husband woke up with a terrible t...,{'source': 'EMPATHY_test'},"[{'turn_id': 0, 'agent_id': 0, 'text': 'My hus...",True,f06e80eb-5e51-4035-8048-2379e79d99c9,False,None,[],NaN,[agent 0: My husband woke up with his face swo...,0,"agent 0: Not yet, but I was his bedside nurse...","Not yet, but I w

In [295]:
df_phase2  = pd.DataFrame(data = rows)
df_phase2['label_change'] = (df_phase2['label'] != df_phase2['original_label']).astype(int)

In [296]:
label_change_stats = (df_phase2.groupby(['task', 'test'])
                     .agg({
                         'label_change': ['size','sum', 'mean' ]  # mean for rate, sum for mismatches, size for total
                     })
                     .round(4))

# Flatten column names and reset index
label_change_stats.columns = ['total_samples', 'samples_with_label_change' ,'change_rate' ]
label_change_stats = label_change_stats.reset_index()

# Convert rate to percentage
label_change_stats['change_rate'] = label_change_stats['change_rate'] * 100

# Sort by change rate in descending order (optional)
label_change_stats = label_change_stats.sort_values('change_rate', ascending=False)

print("Label Change Statistics by Task and Test:")
print(label_change_stats)

Label Change Statistics by Task and Test:
                             task              test  total_samples  \
0          coreference_resolution  grammatical_role            114   
5              sentiment_analysis          negation            158   
1          coreference_resolution          negation            140   
3  dialog_contradiction_detection          negation            135   
2  dialog_contradiction_detection  grammatical_role             90   
4              sentiment_analysis  grammatical_role             73   

   samples_with_label_change  change_rate  
0                         94        82.46  
5                         84        53.16  
1                         68        48.57  
3                         56        41.48  
2                         30        33.33  
4                         11        15.07  


In [297]:
label_change_stats

,task,test,total_samples,samples_with_label_change,change_rate
0,coreference_resolution,grammatical_role,114,94,82.46
5,sentiment_analysis,negation,158,84,53.16
1,coreference_resolution,negation,140,68,48.57
3,dialog_contradiction_detection,negation,135,56,41.48
2,dialog_contradiction_detection,grammatical_role,90,30,33.33
4,sentiment_analysis,grammatical_role,73,11,15.07


In [298]:
df_annotator_group = df_annotator.groupby('annotator_id').mean().reset_index()
df_annotator_group['success_rate_control_only'].value_counts()

success_rate_control_only
1.000000    93
0.666667    44
0.333333     8
0.000000     4
Name: count, dtype: int64

In [299]:
filter_annotators = list(df_annotator_group.loc[df_annotator_group['success_rate_control_only'] < 0.5]['annotator_id'])

In [300]:
df_subtask_filter

,task,test,subtest,total,retain
0,coreference_resolution,grammatical_role,grammatical_role,20,23
1,sentiment_analysis,grammatical_role,grammatical_role,20,23
2,dialog_contradiction_detection,grammatical_role,grammatical_role,15,18
3,dialog_contradiction_detection,grammatical_role,grammatical_role,15,18
4,coreference_resolution,grammatical_role,grammatical_role,20,23
5,dialog_contradiction_detection,grammatical_role,grammatical_role,15,18
6,coreference_resolution,grammatical_role,grammatical_role,20,23
7,coreference_resolution,grammatical_role,grammatical_role,20,23
8,dialog_contradiction_detection,grammatical_role,grammatical_role,15,18
9,dialog_contradiction_detection,grammatical_role,grammatical_role,15,18


In [301]:
df_retain_task.to_csv('df_data_phase2.csv',index = None)

In [302]:
df_phase2['task']

,original_question,task,test,subtest,label,explanation,original_label,label_change
0,"{'index_in_phase1_annotated_data': 29, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Vivan,him rather than himself so clearly means Lakshman,Lakshman,1
1,"{'index_in_phase1_annotated_data': 33, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Steve,Jose could have hired Steve because Jose is a ...,Jose,1
2,"{'index_in_phase1_annotated_data': 36, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Wanda,"Regina is asking Wanda about an opinion, so it...",Regina,1
3,"{'index_in_phase1_annotated_data': 23, 'text':...",coreference_resolution,grammatical_role,grammatical_role,The attackers,The journalists were waiting so couldn't arrive,the journalists,1
4,"{'index_in_phase1_annotated_data': 30, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Stephanie,Theresa is dead so Logan wouldn't be talking t...,Theresa,1
...,...,...,...,...,...,...,...,...
705,"{'index_in_original_testset': 2179, 'conversat...",dialog_contradiction_detection,negation,,No,Even though this statement looks like a contra...,No,0
706,"{'index_in_original_testset': 42, 'conversatio...",dialog_contradiction_detection,negation,,No,It appears that the statements are not contrad...,Yes,1
707,"{'index_in_original_testset': 2953, 'conversat...",dialog_contradiction_detection,negation,,No,Complete thought no contradiction noted.,No,0
708,"{'index_in_original_testset': 3382, 'conversat...",dialog_contradiction_detection,negation,,No,It appear that person B likes horses. I would ...,No,0


In [303]:
df_phase2.to_csv('df_data_phase2.csv',index = None)

In [304]:
df_phase2_coref = df_phase2[df_phase2['task'] == 'coreference_resolution']

In [308]:
df_phase2_coref['original_question'][2]

{'index_in_phase1_annotated_data': 36,
 'text': 'Regina asks Wanda if she wants to go with him, but Wanda does not want to leave Owuor.',
 'pronoun': 'she',
 'candidates': ['Wanda', 'Regina'],
 'label': 'unknown',
 'label_antecedent': 'n/a',
 'is_control_question': False,
 'source': 'phase1 annotation'}

In [305]:
rows = []
for i, row in df_phase2_coref.iterrows():
    test = row['test']
    

,original_question,task,test,subtest,label,explanation,original_label,label_change
0,"{'index_in_phase1_annotated_data': 29, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Vivan,him rather than himself so clearly means Lakshman,Lakshman,1
1,"{'index_in_phase1_annotated_data': 33, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Steve,Jose could have hired Steve because Jose is a ...,Jose,1
2,"{'index_in_phase1_annotated_data': 36, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Wanda,"Regina is asking Wanda about an opinion, so it...",Regina,1
3,"{'index_in_phase1_annotated_data': 23, 'text':...",coreference_resolution,grammatical_role,grammatical_role,The attackers,The journalists were waiting so couldn't arrive,the journalists,1
4,"{'index_in_phase1_annotated_data': 30, 'text':...",coreference_resolution,grammatical_role,grammatical_role,Stephanie,Theresa is dead so Logan wouldn't be talking t...,Theresa,1
...,...,...,...,...,...,...,...,...
655,"{'index_in_phase1_annotated_data': 10, 'text':...",coreference_resolution,negation,,Cannot Determine — Insufficient Information,He could be either,Steven,1
656,"{'index_in_phase1_annotated_data': 13, 'text':...",coreference_resolution,negation,,Tony,Sentence subject is Tony,Jeff,1
657,"{'index_in_phase1_annotated_data': 8, 'text': ...",coreference_resolution,negation,,Dina,Could be either,Dina,0
658,"{'index_in_phase1_annotated_data': 5, 'text': ...",coreference_resolution,negation,,Cannot Determine — Insufficient Information,Bat or ball could be it,The bat,1


In [422]:
df_coref_task = df_coref.groupby('test')

In [429]:
for group, frame in df_coref_task:
    to_write = '../data_after_phase2/thinh/' + group + '.json'
    obj = frame.to_dict(orient='records')
    print(obj)
    with open(to_write,'w') as f:
        json.dump(obj, f)
    # break

[{'original_text': 'Lakshman asked Vivan to get him some ice cream because he was hot.', 'original_candidates': ['Lakshman', 'Vivan'], 'original_pronoun': 'he', 'original_label': 0, 'modified_text': 'Vivan asked Lakshman to get him some ice cream because he was hot.', 'type': 'grammatical_role', 'modified_candidates': ['Vivan', 'Lakshman'], 'modified_pronoun': 'he', 'modified_label': np.str_('Vivan'), 'test': 'grammatical_role'}, {'original_text': 'Steve hired Jose because he was a hard worker.', 'original_candidates': ['Steve', 'Jose'], 'original_pronoun': 'he', 'original_label': 1, 'modified_text': 'Jose hired Steve because he was a hard worker.', 'type': 'grammatical_role', 'modified_candidates': ['Jose', 'Steve'], 'modified_pronoun': 'he', 'modified_label': np.str_('Steve'), 'test': 'grammatical_role'}, {'original_text': 'Wanda asks Regina if she wants to go with him , but Regina does not want to leave Owuor .', 'original_candidates': ['Wanda', 'Regina'], 'original_pronoun': 'she',

In [430]:
df_sentiment_task = df_sentiment.groupby('test')
for group, frame in df_sentiment_task:
    to_write = '../data_after_phase2/yulia/' + group + '.json'
    obj = frame.to_dict(orient='records')
    print(obj)
    with open(to_write,'w') as f:
        json.dump(obj, f)
    # break

[{'idx': 598, 'original_text': "nonsensical , dull `` cyber-horror '' flick is a grim , hollow exercise in flat scares and bad acting . ", 'label': 0, 'modified_text': "Flat scares and bad acting, dull ``cyber-horror'' flick is a grim, hollow exercise in nonsensical.", 'type': 'grammatical_role', 'modified_label': np.str_('Negative'), 'test': 'grammatical_role'}, {'idx': 787, 'original_text': 'a valueless kiddie paean to pro basketball underwritten by the nba . ', 'label': 0, 'modified_text': 'a valueless kiddie paean to the nba underwritten by pro basketball.', 'type': 'grammatical_role', 'modified_label': np.str_('Negative'), 'test': 'grammatical_role'}, {'idx': 546, 'original_text': 'on the heels of the ring comes a similarly morose and humorless horror movie that , although flawed , is to be commended for its straight-ahead approach to creepiness . ', 'label': 1, 'modified_text': 'on the heels of a similarly morose and humorless horror movie comes the ring that, although flawed, is

In [431]:
df_dialog_task = df_dialog.groupby('test')
for group, frame in df_dialog_task:
    to_write = '../data_after_phase2/rongxin/' + group + '.json'
    obj = frame.to_dict(orient='records')
    print(obj)
    with open(to_write,'w') as f:
        json.dump(obj, f)
    # break

[{'conversation_id': 'e4426e3f-dec2-4485-8b08-770e8f2707f8##hit:10362_conv:20724#truncated#5', 'agents': {'1': {'is_human': True, 'persona_lines': ''}, '0': {'is_human': True, 'persona_lines': ''}}, 'conversation_contexts': 'sentimental ## I recently remembered a sweet memory of my aunt who recently passed away. She would give hundreds of dollars of school supplies to kids in need each year.', 'auxiliary': {'source': 'EMPATHY_test'}, 'turns': [{'turn_id': 0, 'agent_id': 0, 'text': 'I recently remembered a sweet memory of my aunt who recently passed away.', 'turn_context': '', 'auxiliary': {'contradiction': None}}, {'turn_id': 1, 'agent_id': 1, 'text': 'what was it?', 'turn_context': '', 'auxiliary': {'contradiction': None}}, {'turn_id': 2, 'agent_id': 0, 'text': 'She would give hundreds of dollars of school supplies to kids in need each year.', 'turn_context': '', 'auxiliary': {'contradiction': None}}, {'turn_id': 3, 'agent_id': 1, 'text': "wow,  that's an amazing woman. did she rub of